In [1]:
import os, sys, time
import numpy as np
import pydicom as dcm

In [250]:
PATH = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/'
list_patients_to_sort = [43]

In [240]:
file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
file_list.sort()

In [241]:
list_models = []
model_dict = {'Patient Verification':'CBCT', 'Brilliance Big Bore':'CT', 'ARIA RadOnc':'RT'}

def basic_sort_files_by_date(list_images):
    for file in list_images:
        d = dcm.read_file(PATH + file)
        man_model = d.ManufacturerModelName
        
        try:
            dicom_type = model_dict[man_model]
        except:
            dicom_type = man_model
            
        if man_model not in list_models:
            list_models.append(man_model)
        
        try: 
            new_path = PATH + d.AcquisitionDate
        except:
            new_path = PATH + d.StudyDate
            
        if not os.path.exists(new_path):
            os.system("sudo mkdir " + new_path)
            print("Created Directory" + new_path)
        os.system("sudo mv " + PATH+file +" " + new_path+"/"+file)
        
        file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]

In [242]:
def find_RT_dicoms(file_list):
    list_RT = []
    list_RE = []
    list_RS = []
    list_RI = []
    list_RD = []
    other = []

    for file in file_list:
        if 'RT' in file:
            list_RT.append(file)
        elif 'RE' in file:
            list_RE.append(file)
        elif 'RS' in file:
             list_RS.append(file)
        elif 'RI' in file:
            list_RI.append(file)
        elif 'RD' in file:
            list_RD.append(file)
        elif 'CT' not in file:
            other.append(file)
            
    print("RT: " , len(list_RT))
    print("RS: " , len(list_RS))
    print("RI: " , len(list_RI))
    print("RE: ", len(list_RE))
    print("RD: ", len(list_RD))
    print("other: " , len(other))
        

In [466]:
dict_class_UID = {'1.2.840.10008.5.1.4.1.1.2': 'CT', '1.2.840.10008.5.1.4.1.1.481.1': 'RI', '1.2.840.10008.5.1.4.1.1.4': 'MR', '1.2.840.10008.5.1.4.1.1.128':'PET'}

In [285]:
def remove_RI_RT_files(file_list, PATH):
    RI_count = 0
    RT_count = 0
    RE_count = 0
    
    RT_path = PATH + "RT"
    if not os.path.exists(RT_path):
        os.system("sudo mkdir " + RT_path)
        print("Created directory "+RT_path)

    RI_path = PATH + "RI"
    if not os.path.exists(RI_path):
        os.system("sudo mkdir " + RI_path)
        print("Created directory "+RI_path)
    
    for file in file_list:
        if 'RT' in file:
            os.system("sudo mv " + PATH+file +" " + RT_path+"/"+file)
            RT_count += 1
        elif 'RI' in file:
            os.system("sudo mv " + PATH+file +" " + RI_path+"/"+file)
            RI_count += 1
        elif 'RE' in file:
            RE_class = dcm.read_file(PATH+file).ReferencedSeriesSequence[0].ReferencedInstanceSequence[0].ReferencedSOPClassUID
            if dict_class_UID[RE_class] == 'RI':
                os.system("sudo mv " + PATH+file +" " + RI_path+"/"+file)
                RE_count += 1
    print("----------------------------------------")
    print("Files moved: ", RT_count, " RT, ", RI_count, " RI, ",RE_count, " RE")
    print("----------------------------------------")
#     file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
#     print("Files remaining: ", len(file_list))
                
                
            

In [461]:
def sort_image_files_by_RS(file_list, PATH):
    uid_dict = {}
#     uid_dict = {'1.2.246.352.221.49847224801070330484778779483479796917': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_12a', '1.2.246.352.221.57385370530185751817191118715568998795': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/CT_24_JAN_2019', '1.2.246.352.221.57054762373955765701474087700179252884': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_1a', '1.2.246.352.221.56671428375338271441765970661995810219': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_17a', '1.2.246.352.221.490045044740798928510355609447538608810': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_10a', '1.2.246.352.221.56721825157915522053330298820337806494': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_21a', '1.2.246.352.221.46723870751625912794474175760914248343': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_32a', '1.2.246.352.221.48931053109986545514076295059970867633': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_5a', '1.2.246.352.221.481871037131647406617633242931867976606': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_24a', '1.2.246.352.221.539972804958439514812961756058331852966': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_4a', '1.2.246.352.221.49278418537032161262708472516144234165': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_26a', '1.2.246.352.221.53945374536270493755238178882923194501': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_34a', '1.2.246.352.221.50274537895075518205025259901270353579': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_29a', '1.2.246.352.221.529351696225906492913624223510847855509': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_19a', '1.2.246.352.221.57257429504068209995126461117779225510': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_14a', '1.2.246.352.221.49078747694847591456067450471793767568': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/43/kV_CBCT_7a'}
    list_RE = []
    list_RS = []
    list_RD = []
    CT_count = 0
    other = []
    
    for file in file_list:
        if 'RE' in file:
            list_RE.append(file)
        elif 'RS' in file:
             list_RS.append(file)
        elif 'RD' in file:
            list_RD.append(file)
        elif 'CT' not in file:
            other.append(file)
            
    for file in list_RS:
        if 'RS' in file:
            d = dcm.read_file(PATH+file)
            
            new_path = PATH + d.StructureSetDate + "_" +  d.StructureSetLabel
#             print(new_path)
            if not os.path.exists(new_path):
                os.system("sudo mkdir " + new_path)
                print("Created directory "+new_path)
        
            os.system("sudo mv " + PATH+file +" " + new_path+"/"+file)
    
            frame_of_reference_uid = d.ReferencedFrameOfReferenceSequence[0].FrameOfReferenceUID
            uid_dict.update({frame_of_reference_uid: new_path})
#             print("RS Series:", d.SeriesInstanceUID)
        
            
            for img in d.ReferencedFrameOfReferenceSequence[0].RTReferencedStudySequence[0].RTReferencedSeriesSequence[0].ContourImageSequence:
                uid = img.ReferencedSOPInstanceUID 
                os.system("sudo mv " + PATH+"CT."+uid+".dcm" +" " + new_path+"/"+"CT."+uid+".dcm")
                CT_count += 1

    
    for file in list_RE:
        d = dcm.read_file(PATH+file)
        try:
            frame_of_reference_uid = d.RegistrationSequence[1].FrameOfReferenceUID
        except:
            print("HERE")
            frame_of_reference_uid = d.FrameOfReferenceUID
#         print(frame_of_reference_uid)
#         print(uid_dict)
#         print(uid_dict[frame_of_reference_uid])
        try:
            os.system("sudo mv " + PATH+file +" " + uid_dict[frame_of_reference_uid]+"/"+file)
        except:
            print("could not move file ", file, " with frame of ref uid ",frame_of_reference_uid)


    for file in list_RD:
        d = dcm.read_file(PATH+file)
        frame_of_reference_uid = d.FrameOfReferenceUID
#         print("RD Series:", d.SeriesInstanceUID)
        try:
            os.system("sudo mv " + PATH+file +" " + uid_dict[frame_of_reference_uid]+"/"+file)
        except:
            print("could not move file ", file, " with frame of ref uid ",frame_of_reference_uid)

    
    if len(other) != 0:
        print("Other files not moved")
        for file in other:
            print(file)
    
    print("----------------------------------------")
    print("Files moved: ",CT_count, " CT, ", len(list_RS), " RS, ", len(list_RE), " RE, ",len(list_RD), " RD")
    print("----------------------------------------")
#     file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
#     print("Files remaining: ", len(file_list))

In [471]:
def remove_unneeded_RE_files(PATH):
    file_list = [f for f in os.listdir(patient_path) if os.path.isfile(os.path.join(patient_path, f)) and 'RE' in f]
    for file in file_list:
        d = dcm.read_file(PATH+file)
        class_UID = d.RegistrationSequence[1].ReferencedImageSequence[0].ReferencedSOPClassUID
        if dict_class_UID[class_UID] != 'CT':
            print("Removing "+dict_class_UID[class_UID]+" registration file.")
#             print("sudo rm " + PATH+file)
            os.system("sudo rm " + PATH+file)

In [404]:
# find_RT_dicoms(file_list)

In [405]:
# remove_RI_RT_files(file_list)

In [406]:
# sort_image_files_by_RS(file_list)

In [472]:
def organize_multiple_patients(list_patients, PATH):
    for patient in list_patients:
        print("=======================================")
        print("Patient ", patient)
        patient_path = PATH + str(patient) + "/"
        file_list = [f for f in os.listdir(patient_path) if os.path.isfile(os.path.join(patient_path, f))]
        remove_RI_RT_files(file_list, patient_path)
        file_list = [f for f in os.listdir(patient_path) if os.path.isfile(os.path.join(patient_path, f))]
        sort_image_files_by_RS(file_list, patient_path)
        

In [473]:
remove_unneeded_RE_files('/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/')

Removing MR registration file.
Removing PET registration file.
Removing MR registration file.
Removing MR registration file.


In [443]:
list_patients_to_sort = [81]
organize_multiple_patients(list_patients_to_sort, PATH)

Patient  81
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/RT
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/RI
----------------------------------------
Files moved:  105  RT,  138  RI,  55  RE
----------------------------------------
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190226_kV_CBCT_1a
RS Series: 1.2.246.352.221.545794643994284907217941202424238938259
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190215_kV_CBCT_14a
RS Series: 1.2.246.352.221.506094486406648225710289629501012552368
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190213_kV_CBCT_12a
RS Series: 1.2.246.352.221.487531483639524679311837699953227992757
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190315_kV_CBCT_14a
RS Series: 1.2.246.352.221.530984804866474730410828058139336612763
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/2019

mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.490161031020480724813945943122635944383.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.56356820235666664569110335385142835896.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.520779091039476590513372154149201111719.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.50881546251516614136272142850668502429.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.51664586865704620722885655188071159946.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.462250622409891263116834008327594647198.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveSha

mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.518907236495094854115625110537934816186.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.49240558360775749271815049482851681417.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.498594559901858733713601350549031910569.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.538111317945260807214952862080292336009.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.47304415112486541545652029161809657507.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/CT.1.2.246.352.221.57189707279766491612472414546563271071.dcm': No such file or directory
mv: cannot stat '/mnt/iDriveSha

Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190204_kV_CBCT_6a
RS Series: 1.2.246.352.221.57120706361068857512658815685602646190
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190305_kV_CBCT_6a
RS Series: 1.2.246.352.221.557799239595198921811987216208929932473
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190225_CT_22_FEB_2019
RS Series: 1.2.246.352.221.515346879464128170915911633581892160416
Created directory /mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190130_kV_CBCT_3a
RS Series: 1.2.246.352.221.479125841197731018013801484444593961123
{'1.2.246.352.221.5217924600390167664703954553358945189': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190226_kV_CBCT_1a', '1.2.246.352.221.52690339509943730502468908871279984549': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190215_kV_CBCT_14a', '1.2.246.352.221.563186795087721944514599267832862867072': '/mnt/iDriveShare/Kayla/CBCT_im

{'1.2.246.352.221.5217924600390167664703954553358945189': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190226_kV_CBCT_1a', '1.2.246.352.221.52690339509943730502468908871279984549': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190215_kV_CBCT_14a', '1.2.246.352.221.563186795087721944514599267832862867072': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190213_kV_CBCT_12a', '1.2.246.352.221.5541654755077471498301674366289369274': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190315_kV_CBCT_14a', '1.2.246.352.221.47358213635990794172087397009966433698': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190218_kV_CBCT_15a', '1.2.246.352.221.575328382854147146015618888906357488042': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190228_kV_CBCT_3a', '1.2.246.352.221.544853637922354988014461306329629061537': '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190206_kV_CBCT_8a', '1.2.246.352.221.57480556581869626917886457994141

RD Series: 1.2.246.352.221.46662214434920016061644437831187360439
RD Series: 1.2.246.352.221.5040372473291567266870165455973201547
RD Series: 1.2.246.352.221.51014691828157400082510959814212816524
RD Series: 1.2.246.352.221.54015297472664663767150713691537427113
RD Series: 1.2.246.352.221.523451455201709110815671735096445098649
----------------------------------------
Files moved:  1722  CT,  17  RS,  21  RE,  6  RD
----------------------------------------


In [446]:
test_path = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/81/20190225_CT_22_FEB_2019'
test = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f)) and 'RD' in f]
print(test)
# test_path2 = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/54/20190125_TEST'
# test2 = [f for f in os.listdir(test_path2) if os.path.isfile(os.path.join(test_path2, f))]
# print(test2)


['RD.1.2.246.352.221.54388004773165337695339792520386048149.dcm', 'RD.1.2.246.352.221.50766356216895066391655975358811339957.dcm', 'RD.1.2.246.352.221.54553448380792252658120390802958965680.dcm']


In [452]:
d = dcm.read_file(test_path+"/"+test[0])
d

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 188
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Dose Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.246.352.221.54388004773165337695339792520386048149
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.246.352.70.2.1.160.3
(0002, 0013) Implementation Version Name         SH: 'DCIE 2.2'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20230227'
(0008, 0013) Instance Creation Time              TM: '153913'
(0008, 0016) SOP Class UID                       UI: RT Dose Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.221.54388004773165337695339792520386048149
(0008, 0020